### Notebook for training the hand gesture classification

In [37]:
#Import dependencies

import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots


RANDOM_SEED = 42

In [54]:
#Path selection and definition of constant

dataset = '../assets/dataset/keypoint.csv'
model_save_path = '../assets/models'
tflite_save_path = '../assets/models/tf_LiteModel.tflite'

epoche = 1000
batch_size = 128
num_classes = 6

### Dataset Reading

In [39]:
x_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, train_size=0.70, random_state=RANDOM_SEED)

### Model Building

In [40]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])


model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_6 (Dropout)         (None, 42)                0         
                                                                 
 dense_9 (Dense)             (None, 20)                860       
                                                                 
 dropout_7 (Dropout)         (None, 20)                0         
                                                                 
 dense_10 (Dense)            (None, 10)                210       
                                                                 
 dense_11 (Dense)            (None, 6)                 66        
                                                                 
Total params: 1,136
Trainable params: 1,136
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
model_save_path+'/modello.h5',verbose=1, save_weights_only=False, save_best_only= True, mode='auto')
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

# Model compilation
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_6 (Dropout)         (None, 42)                0         
                                                                 
 dense_9 (Dense)             (None, 20)                860       
                                                                 
 dropout_7 (Dropout)         (None, 20)                0         
                                                                 
 dense_10 (Dense)            (None, 10)                210       
                                                                 
 dense_11 (Dense)            (None, 6)                 66        
                                                                 
Total params: 1,136
Trainable params: 1,136
Non-trainable params: 0
_________________________________________________________________


### Training

In [42]:
history = model.fit(
    x_train,
    y_train,
    epochs=epoche,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[cp_callback, es_callback]
)

model.save(model_save_path+'/model.h5')
model.save(model_save_path+'/model.h5py')

Epoch 1/1000
57/99 [================>.............] - ETA: 0s - loss: 1.8003 - accuracy: 0.1937 
Epoch 1: val_loss improved from inf to 1.47956, saving model to ../assets/models/modello.h5
99/99 [==============================] - 1s 3ms/step - loss: 1.7409 - accuracy: 0.2249 - val_loss: 1.4796 - val_accuracy: 0.5894
Epoch 2/1000
54/99 [===============>..............] - ETA: 0s - loss: 1.4887 - accuracy: 0.3866
Epoch 2: val_loss improved from 1.47956 to 1.05631, saving model to ../assets/models/modello.h5
99/99 [==============================] - 0s 2ms/step - loss: 1.4251 - accuracy: 0.4101 - val_loss: 1.0563 - val_accuracy: 0.7065
Epoch 3/1000
54/99 [===============>..............] - ETA: 0s - loss: 1.2081 - accuracy: 0.5055
Epoch 3: val_loss improved from 1.05631 to 0.78684, saving model to ../assets/models/modello.h5
99/99 [==============================] - 0s 2ms/step - loss: 1.1586 - accuracy: 0.5390 - val_loss: 0.7868 - val_accuracy: 0.9272
Epoch 4/1000
55/99 [===============>....

In [43]:
for key in history.history:
    print(key)

loss
accuracy
val_loss
val_accuracy


In [44]:
# Model evaluation

val_loss, val_acc = model.evaluate(x_test, y_test, batch_size)

43/43 [==============================] - 0s 1ms/step - loss: 0.0269 - accuracy: 0.9980


In [45]:
# Loading saved model

model = tf.keras.models.load_model(model_save_path+'/model.h5')

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_6 (Dropout)         (None, 42)                0         
                                                                 
 dense_9 (Dense)             (None, 20)                860       
                                                                 
 dropout_7 (Dropout)         (None, 20)                0         
                                                                 
 dense_10 (Dense)            (None, 10)                210       
                                                                 
 dense_11 (Dense)            (None, 6)                 66        
                                                                 
Total params: 1,136
Trainable params: 1,136
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Save as a model dedicated to inference
model.save(model_save_path+'/model_include_optimize.h5', include_optimizer=False)

# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /var/folders/n8/sgss1v152672p_crqrvdd3_m0000gn/T/tmpdccivqcn/assets


INFO:tensorflow:Assets written to: /var/folders/n8/sgss1v152672p_crqrvdd3_m0000gn/T/tmpdccivqcn/assets


6408